In [1]:
import pandas as pd
import numpy as np

In [2]:
charterdf = pd.read_pickle('../../nowdata/charters_2015.pkl')

In [3]:
list(charterdf.columns)

['NCESSCH',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AM',
 'AMALM',
 'AMALF',
 'AS',
 'ASALM',
 'ASALF',
 'HI',
 'HIALM',
 'HIALF',
 'BL',
 'BLALM',
 'BLALF',
 'WH',


In [4]:
charterdf.shape

(12877, 680)

In [5]:
df_URLs = pd.read_csv("../../nowdata/backups/charter_URLs_2016.csv")  # Holds URL for each school in 'URL' column
df_URLs = df_URLs[["NCESSCH", "URL"]]

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,41,48,52,81,82,100,130,186,213) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
charter_df = pd.merge(charterdf, df_URLs, how="outer", on="NCESSCH") # Add 'URL' column from df_URLs to charter_df, joining on 'NCESSCH'. how='outer' means keep all keys and rows from both DFs, in this case resulting in a full list of all charter schools and URLs

In [7]:
# Making sure URL column is kept
charter_df['URL'].head()

0    http://www.maef.net/our-work/programs/accel/#p...
1                             http://ayaprun.lksd.org/
2               https://www.kgbsd.org/ketchikancharter
3                        http://www.tongassschool.org/
4                         https://aquarian.asdk12.org/
Name: URL, dtype: object

In [8]:
df_URLs["NCESSCH"] = df_URLs["NCESSCH"].astype(float)# Convert to float type (just in case)
charter_df["NCESSCH"] = charter_df["NCESSCH"].astype(float) # Trim down to just the variables we want: unique ID and URL


In [ ]:
charter_df.iloc[17999]

In [25]:
charter_df = charter_df.dropna(subset = ['WEBTEXT']) #dropping NaN values in WEBTEXT -- merging creates NaN values

In [26]:

missing1 = charter_df[charter_df["WEBTEXT"].apply(len) == 0] # Keep rows where NO WEBTEXT (0 pages) was gathered (i.e., scrape failed)

charter_df = charter_df[charter_df["WEBTEXT"].apply(len) > 0] # Drop rows where NO WEBTEXT (0 pages) was gathered (i.e., scrape failed)
missing2 = charter_df[charter_df["WEBTEXT"].apply(lambda school: len(school[0][3])) == 0] # Keep rows where first scraped page is empty
missing3 = charter_df[charter_df["WEBTEXT"].apply(lambda school: sum([len(page[3]) for page in school])) == 0] # Keep rows where all pages are empty

missingdf = pd.concat([missing1, missing2, missing3])

In [27]:
missingdf.shape

(5527, 681)

## Filtering Data 

Determine rows based on if school is not scrapped yet and still open.

Not scrapped: `WEBTEXT` length == 0  
School opened: look at latest SY_STATUS [`SY_STATUS15`]  
    `open:` SY_STATUS15 == 1,3,4,5 or 8  
    `closed:` SY_STATUS15 == 2,6 or 7

In [46]:
#filtered_WEBTEXT = (charterdf['WEBTEXT'].str.len() < 1) 

#no_urls = charterdf.loc[filtered_WEBTEXT]
#no_urls[['SY_STATUS','WEBTEXT']].head()

In [30]:
filtered_STATUS = (missingdf['SY_STATUS15'] == 1) | (missingdf['SY_STATUS15'] == 3) | (missingdf['SY_STATUS15'] == 4) | (missingdf['SY_STATUS15'] == 5) | (missingdf['SY_STATUS15'] == 8)

#filtered_STATUS
charter_df = missingdf.loc[filtered_STATUS]
charter_df

,NCESSCH,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,...,SCH_FTESERVICES_NUR,SCH_FTESERVICES_PSY,SCH_FTESERVICES_SOC,DISCIPLINE_COUNT,INQUIRY_COUNT,DISC_STR,INQU_STR,NUMWORDS,NUMPAGES,URL
1,2.000010e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,http://ayaprun.lksd.org/
5,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,https://familypartnership.asdk12.org/
12,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.80,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,https://rilkeschule.asdk12.org/
16,2.003900e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,http://nome.nosd.schoolaccess.net/~acsa/
32,2.006000e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,http://ekc.k12northstar.org/
146,4.000740e+10,Kingman Academy of Learning,1344.0,KINGMAN ACADEMY OF LEARNING - HIGH SCHOOL,AZ,5.0,http://kaolaz.org/,1.0,AZ,CMO,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,http://kaolaz.org/calendar.html
149,4.000770e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,NaN
266,4.001250e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0
304,4.001410e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.0,0.0,NaN,NaN,0.0,0.0,0


In [31]:
len(df_URLs)

7400

In [32]:
charter_df.head() # Just double-checking the view

,NCESSCH,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,...,SCH_FTESERVICES_NUR,SCH_FTESERVICES_PSY,SCH_FTESERVICES_SOC,DISCIPLINE_COUNT,INQUIRY_COUNT,DISC_STR,INQU_STR,NUMWORDS,NUMPAGES,URL
1,2.000010e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,http://ayaprun.lksd.org/
5,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,https://familypartnership.asdk12.org/
12,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,https://rilkeschule.asdk12.org/
16,2.003900e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,http://nome.nosd.schoolaccess.net/~acsa/


In [33]:
list(charter_df)

['NCESSCH',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AM',
 'AMALM',
 'AMALF',
 'AS',
 'ASALM',
 'ASALF',
 'HI',
 'HIALM',
 'HIALF',
 'BL',
 'BLALM',
 'BLALF',
 'WH',


In [34]:
charter_df = charter_df[["NCESSCH", "URL"]] # The only variables we need for web-crawling
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") # Getting rid of duplicate NCESSCH columns (shouldn't be looking over the same school twice)

In [35]:
charter_df.shape

(1009, 2)

In [36]:
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") 

In [37]:
charter_df.columns = ['NCESSCH', 'URL']
charter_df.head() # View of the data

,NCESSCH,URL
1,2.000010e+10,http://ayaprun.lksd.org/
5,2.001800e+10,https://familypartnership.asdk12.org/
12,2.001800e+10,https://rilkeschule.asdk12.org/
16,2.003900e+10,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,http://nome.nosd.schoolaccess.net/~acsa/


In [38]:
charter_df.shape

(1009, 2)

In [43]:
# Prianka's attempt to convert the dataframe to a CSV.
charter_df.to_csv('../data/charter_urls_2016_unscraped.csv')
charter_df.to_csv('../../scrapy-cluster/kafka-monitor/charter_urls_2016_unscraped.csv')
# It looks like the conversion happened fine, just not sure where the file outputted to.

End result: save the resulting data frame as a CSV named charter_URLs_2016_unscraped.csv in web_scraping/data and in scrapy-cluster/kafka-monitor/